In [1]:
import yfinance as yf

# Define the ticker symbol
ticker_symbol = "RELIANCE.NS"

# Download historical data
reliance_data = yf.download(ticker_symbol, start="1900-01-01")

# Save to CSV file
reliance_data.to_csv("reliance_historical_data.csv")

print("Historical data for Reliance Industries has been downloaded and saved to 'reliance_historical_data.csv'.")


[*********************100%%**********************]  1 of 1 completed

Historical data for Reliance Industries has been downloaded and saved to 'reliance_historical_data.csv'.


In [2]:
reliance_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1996-01-01,14.638248,14.716795,14.541850,14.691803,9.302442,52060684
1996-01-02,14.656100,14.727506,14.470444,14.577553,9.230104,84371654
1996-01-03,14.816763,15.491551,14.656100,14.688232,9.300180,104661939
1996-01-04,14.548991,14.595404,14.356194,14.552561,9.214277,108450132
1996-01-05,14.495436,14.495436,14.327632,14.452592,9.150981,83354233
...,...,...,...,...,...,...
2024-07-22,3074.500000,3075.000000,2998.550049,3001.350098,3001.350098,9837188
2024-07-23,3002.000000,3011.500000,2926.000000,2975.800049,2975.800049,9026022
2024-07-24,2970.000000,3019.000000,2952.800049,2991.399902,2991.399902,5956681


In [3]:
import yfinance as yf
import pandas as pd
import sqlite3

# Define the ticker symbol
ticker_symbol = "RELIANCE.NS"

# Download historical data
reliance_data = yf.download(ticker_symbol, start="1900-01-01")

# Create a connection to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('reliance_data.db')
cursor = conn.cursor()

# Create a table for the historical data
cursor.execute('''
CREATE TABLE IF NOT EXISTS reliance (
    Date TEXT PRIMARY KEY,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Adj_Close REAL,
    Volume INTEGER
)
''')

# Convert the DataFrame to a SQL table
reliance_data.to_sql('reliance', conn, if_exists='replace', index_label='Date')

# Commit changes and close the connection
conn.commit()
conn.close()

print("Historical data for Reliance Industries has been downloaded and saved to 'reliance_data.db'.")



[*********************100%%**********************]  1 of 1 completed

Historical data for Reliance Industries has been downloaded and saved to 'reliance_data.db'.


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

[*********************100%%**********************]  1 of 1 completed


In [6]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine, Column, Float, Integer, String, Date
from sqlalchemy.orm import declarative_base, sessionmaker
import plotly.express as px

# Ensure nbformat and ipython are installed and up to date
!pip install nbformat ipython --upgrade

# Define the SQLAlchemy base and model
Base = declarative_base()

class Reliance(Base):
    __tablename__ = 'reliance'
    Date = Column(Date, primary_key=True)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Adj_Close = Column(Float)
    Volume = Column(Integer)

# Create an engine and session
engine = create_engine('sqlite:///reliance_data.db')

# Drop the existing table if it exists
Base.metadata.drop_all(engine)

# Create the table again
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Define the ticker symbol
ticker_symbol = "RELIANCE.NS"

# Download historical data
reliance_data = yf.download(ticker_symbol, start="1900-01-01")

# Convert the Date index to a column
reliance_data.reset_index(inplace=True)

# Rename the 'Adj Close' column to 'Adj_Close' to match the database schema
reliance_data.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)

# Save data to the database
for index, row in reliance_data.iterrows():
    reliance_record = Reliance(
        Date=row['Date'],
        Open=row['Open'],
        High=row['High'],
        Low=row['Low'],
        Close=row['Close'],
        Adj_Close=row['Adj_Close'],
        Volume=row['Volume']
    )
    session.merge(reliance_record)

session.commit()

# Fetch data from the database
reliance_data_db = pd.read_sql(session.query(Reliance).statement, session.bind)

# Convert the 'Date' column to datetime format
reliance_data_db['Date'] = pd.to_datetime(reliance_data_db['Date'])

# Plot the close price using Plotly
fig = px.line(reliance_data_db, x='Date', y='Close', title='Reliance Industries Close Price Over Time')
fig.show()

# Close the session
session.close()


[*********************100%%**********************]  1 of 1 completed


OperationalError: (sqlite3.OperationalError) no such column: reliance.Adj_Close
[SQL: SELECT reliance."Date" AS "reliance_Date", reliance."Open" AS "reliance_Open", reliance."High" AS "reliance_High", reliance."Low" AS "reliance_Low", reliance."Close" AS "reliance_Close", reliance."Adj_Close" AS "reliance_Adj_Close", reliance."Volume" AS "reliance_Volume" 
FROM reliance 
WHERE reliance."Date" = ?]
[parameters: ('1996-01-01',)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)